In [1]:
%matplotlib inline

In [2]:
import tensorflow as tf
import data_provider
import cacac_train
import models
import numpy as np
import matplotlib.pyplot as plt

from menpo.visualize import print_dynamic
slim = tf.contrib.slim

In [3]:
nogpu_config = tf.ConfigProto(
    # Do not use a GPU device
    device_count = {'GPU': 0}
)

sess = tf.Session(config=nogpu_config)

In [4]:
sess = tf.Session()

In [5]:
audio, ground_truth, _ = data_provider.get_split('urtic', 'train', batch_size=1)

In [6]:
tf.train.start_queue_runners(sess=sess)

[<Thread(Thread-4, started daemon 140087898646272)>,
 <Thread(Thread-5, started daemon 140087890253568)>,
 <Thread(Thread-6, started daemon 140087539005184)>,
 <Thread(Thread-7, started daemon 140087530612480)>,
 <Thread(Thread-8, started daemon 140087522219776)>,
 <Thread(Thread-9, started daemon 140087513827072)>]

In [7]:
with slim.arg_scope([slim.batch_norm, slim.layers.dropout],
                    is_training=False):
    prediction = models.get_model('audio')(audio)

In [8]:
variables_to_restore = slim.get_variables_to_restore()
saver = tf.train.Saver(variables_to_restore)
model_path = slim.evaluation.tf_saver.get_checkpoint_state('ckpt/train').model_checkpoint_path
saver.restore(sess, model_path)
print(model_path)

ckpt/train/model.ckpt-0


In [9]:
accuracy = tf.reduce_mean(
    tf.to_float(tf.argmax(ground_truth, 1) == tf.argmax(prediction, 1)))

In [78]:
sess.run(tf.to_float(tf.equal(tf.argmax(ground_truth, 1), tf.argmax(prediction, 1))))

array([ 1.], dtype=float32)

In [ ]:
preds = []
gts = []

num_samples = 3594

for i in (range(num_samples)):
    a, b = sess.run([tf.argmax(prediction, 1), tf.argmax(ground_truth, 1)])
    preds.append(a[0])
    gts.append(b[0])
    
    print_dynamic("{}/{}, ac: {:.2f}%, rnd: {:.2f}%".format(
            i, num_samples, np.mean(np.array(preds) == np.array(gts)), np.array(gts).mean()))

731/3594, ac: 0.60%, rnd: 0.60%                                                 

## tf.random_normal((640,), stddev=.2)

In [15]:
(np.mean(np.array(preds) == np.array(gts)) /  np.array(gts).mean()) / 2

0.52427564604541899